In [1]:
# Google Calendar API integration for appointment scheduling
PROJECT_ID = "avalon-conversational-agent"

from google.cloud import storage
import json
from datetime import datetime, timedelta

In [2]:
# Install Google Calendar API client
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib -q

In [1]:
# Set up Google Calendar API
from googleapiclient.discovery import build
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
SERVICE_ACCOUNT_FILE = 'avalon-conversational-agent-7570ac8b695d.json'
CALENDAR_ID = '29783fcaaaf3b125207f80638a051bdbbe856038631f21f724f8509f9d099cb3@group.calendar.google.com'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

print("Calendar API connected")

Calendar API connected


In [6]:
# Format times nicely
from dateutil import parser

def get_events_clean(days_ahead=30):
    now = datetime.utcnow()
    time_min = now.isoformat() + 'Z'
    time_max = (now + timedelta(days=days_ahead)).isoformat() + 'Z'
    
    events_result = service.events().list(
        calendarId=CALENDAR_ID,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    
    events = events_result.get('items', [])
    
    print(f"Found {len(events)} events in next {days_ahead} days:\n")
    for event in events:
        summary = event.get('summary', 'Busy')
        start_raw = event['start'].get('dateTime', event['start'].get('date'))
        end_raw = event['end'].get('dateTime', event['end'].get('date'))
        
        if 'T' in start_raw:
            start = parser.parse(start_raw).strftime('%b %d, %I:%M %p')
            end = parser.parse(end_raw).strftime('%I:%M %p EST')
            print(f"  {summary}: {start} - {end}")
        else:
            print(f"  {summary}: {start_raw} (all day)")
    
    return events

events = get_events_clean()

Found 2 events in next 30 days:

  busy: 2025-12-15 (all day)
  busy: Dec 16, 05:15 AM - 09:45 AM EST


In [8]:
# Find available slots in 15-minute intervals
def get_available_slots(days_ahead=30, duration_minutes=60):
    OFFICE_START = 8
    OFFICE_END = 17
    WORK_DAYS = [0, 1, 2, 3]  # Mon-Thu
    
    now = datetime.now(est)
    time_min = now.isoformat()
    time_max = (now + timedelta(days=days_ahead)).isoformat()
    
    events_result = service.events().list(
        calendarId=CALENDAR_ID,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    busy_periods = []
    for event in events:
        start_raw = event['start'].get('dateTime', event['start'].get('date'))
        end_raw = event['end'].get('dateTime', event['end'].get('date'))
        
        if 'T' in start_raw:
            busy_periods.append((parser.parse(start_raw), parser.parse(end_raw)))
        else:
            start_date = parser.parse(start_raw)
            busy_periods.append((
                est.localize(datetime.combine(start_date, datetime.min.time())),
                est.localize(datetime.combine(start_date + timedelta(days=1), datetime.min.time()))
            ))
    
    available = []
    current_day = now.date()
    
    for day_offset in range(days_ahead):
        check_date = current_day + timedelta(days=day_offset)
        
        if check_date.weekday() not in WORK_DAYS:
            continue
        
        # 15-minute intervals from 8 AM to 5 PM
        for hour in range(OFFICE_START, OFFICE_END):
            for minute in [0, 15, 30, 45]:
                slot_start = est.localize(datetime.combine(check_date, datetime.min.time().replace(hour=hour, minute=minute)))
                slot_end = slot_start + timedelta(minutes=duration_minutes)
                
                if slot_start < now:
                    continue
                
                is_available = True
                for busy_start, busy_end in busy_periods:
                    if slot_start < busy_end and slot_end > busy_start:
                        is_available = False
                        break
                
                if is_available:
                    available.append(slot_start)
    
    return available

slots = get_available_slots()
print(f"Found {len(slots)} available slots\n")
print("Next 10 available:")
for slot in slots[:10]:
    print(f"  {slot.strftime('%A, %b %d at %I:%M %p')}")

Found 533 available slots

Next 10 available:
  Tuesday, Dec 16 at 09:45 AM
  Tuesday, Dec 16 at 10:00 AM
  Tuesday, Dec 16 at 10:15 AM
  Tuesday, Dec 16 at 10:30 AM
  Tuesday, Dec 16 at 10:45 AM
  Tuesday, Dec 16 at 11:00 AM
  Tuesday, Dec 16 at 11:15 AM
  Tuesday, Dec 16 at 11:30 AM
  Tuesday, Dec 16 at 11:45 AM
  Tuesday, Dec 16 at 12:00 PM


In [9]:
# Check availability for a specific date
def get_slots_for_date(date_str, duration_minutes=60):
    """Get available slots for a specific date. Format: 'Dec 16' or 'December 16'"""
    target = parser.parse(date_str + ' 2025')
    all_slots = get_available_slots(days_ahead=30, duration_minutes=duration_minutes)
    
    matching = [s for s in all_slots if s.date() == target.date()]
    
    if matching:
        print(f"Available on {target.strftime('%A, %b %d')}:")
        for slot in matching:
            print(f"  {slot.strftime('%I:%M %p')}")
    else:
        print(f"No availability on {target.strftime('%A, %b %d')}")
    
    return matching

# Test it
get_slots_for_date("Dec 16")

Available on Tuesday, Dec 16:
  09:45 AM
  10:00 AM
  10:15 AM
  10:30 AM
  10:45 AM
  11:00 AM
  11:15 AM
  11:30 AM
  11:45 AM
  12:00 PM
  12:15 PM
  12:30 PM
  12:45 PM
  01:00 PM
  01:15 PM
  01:30 PM
  01:45 PM
  02:00 PM
  02:15 PM
  02:30 PM
  02:45 PM
  03:00 PM
  03:15 PM
  03:30 PM
  03:45 PM
  04:00 PM
  04:15 PM
  04:30 PM
  04:45 PM


[datetime.datetime(2025, 12, 16, 9, 45, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 10, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 10, 15, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 10, 30, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 10, 45, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 11, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 11, 15, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 11, 30, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 11, 45, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2025, 12, 16, 12, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 datetime.date